In [46]:
from dynamics import *
import numpy as np
import pandas as pd

file_path   = 'bvolmatrix.xlsx'
sheet       = 'STRADDLE'
sheetf      = 'STRIKE'
df          = pd.read_excel(file_path, sheet_name=sheet,header=None)
dff         = pd.read_excel(file_path, sheet_name=sheetf,header=None)
straddlesobs       = pd.DataFrame(df[[x for x in np.arange(4,16)]][3:22])
strikes            = pd.DataFrame(dff[[x for x in np.arange(4,16)]][3:22])
straddlesobs.index     = df[1][3:22].values
strikes.index          = dff[1][3:22].values
straddlesobs.index.name='Expiry'
strikes.index.name     ='Expiry' 
straddlesobs.columns   = [int(x) for x in df.iloc[1][4:16].values]
strikes.columns        = [int(x) for x in dff.iloc[1][4:16].values]
# straddlesobs
sheeta      = 'ARTICLE'
dfff        = pd.read_excel(file_path, sheet_name=sheeta,header=None)
articleobs  = dfff[[x for x in np.arange(4,17)]][3:17]/10000
articleobs.index = dfff[1][3:17].values
articleobs.index.name = 'Expiry'
articleobs.columns = [int(x) for x in dfff.iloc[1][4:17].values]
articleobs


,24,36,48,60,72,84,96,108,120,180,240,300,360
Expiry,,,,,,,,,,,,,
24,0.0095,0.0155,0.0216,0.0273,0.0331,0.0386,0.0437,0.0488,0.0537,0.0746,0.0933,0.1108,0.1270
36,0.0143,0.0222,0.0297,0.0373,0.0447,0.0518,0.0585,0.0651,0.0713,0.0975,0.1221,0.1445,0.1653
48,0.0185,0.0279,0.0371,0.0460,0.0548,0.0632,0.0715,0.0793,0.0865,0.1170,0.1459,0.1723,0.1976
60,0.0221,0.0328,0.0432,0.0534,0.0634,0.0731,0.0824,0.0913,0.0996,0.1338,0.1666,0.1964,0.2250
72,0.0248,0.0367,0.0481,0.0594,0.0705,0.0810,0.0912,0.1008,0.1101,0.1472,0.1832,0.2156,0.2471
84,0.0271,0.0399,0.0523,0.0644,0.0763,0.0876,0.0986,0.1095,0.1200,0.1598,0.1980,0.2331,0.2674
96,0.0290,0.0427,0.0560,0.0690,0.0818,0.0940,0.1058,0.1171,0.1281,0.1705,0.2106,0.2487,0.2853
108,0.0306,0.0451,0.0591,0.0727,0.0861,0.0992,0.1116,0.1236,0.1355,0.1801,0.2227,0.2623,0.3007
120,0.0320,0.0473,0.0620,0.0765,0.0904,0.1041,0.1170,0.1297,0.1427,0.1902,0.2344,0.2755,0.3164


In [117]:
def VasicekOptimizer(initial,mean,reversion,volatility, observedP, observedK):
    straddleVasi = observedP.copy()
    optVasi = Vasicek(initial=initial, mean=mean, reversion=reversion, volatility=volatility)
    # optVasi = Vasicek(initial=initial, mean=0.02, reversion=0.95, volatility=volatility)
    for expiry in observedP.index:
        # Expiry first
        # Getting the right expiry and converting to years
        
        for tenor in observedP.columns:
            # print(expiry,tenor)
            # Creating tenor legs
            T = np.arange(expiry/12, expiry/12+tenor/12+0.5, 0.5)
            S = np.arange(expiry/12, expiry/12+tenor/12+1)
            # Finding the ATM fixed rate
            K = observedK[tenor].loc[expiry]/100

            #Calculating the payer and receiver swpt price
            p = optVasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=K, payer=True)
            r = optVasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=K, payer=False)
            # print(f'Expiry:{expiry}, Tenor:{tenor}, Straddle:{p+r}')
            straddleVasi[tenor].loc[expiry] = (p+r)*10000
            # print(f'Expiry: {expiry}, Tenor: {tenor} fixedrate: {K}')
            # print(f'T={T}')
            # print(f'S={S}')
            # print(f'Price:{p+r}')
    error = ((straddleVasi-observedP)**2).sum().sum()
    print(f'{error}', end='                                                         \r')
    return error,straddleVasi

In [63]:
#Trying to fit all prices (failing)
from scipy.optimize import minimize
EURIBORFIXMARCH22 = 0.03071
Nfeval = 0
def callback(xk):
    global Nfeval
    print(f'Eval: {Nfeval}, x={xk}, func: {VasicekOptimizer(initial=0.03071, mean=xk[0], reversion=xk[1], volatility=xk[2],observedP=straddlesobs, observedK=strikes)[0]}')
    Nfeval += 1
    return True
# callback([0.027832947, 0.11874695, 0.019677])

result = minimize(
    # fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observed=straddlesobs)[0],
    fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observedP=straddlesobs, observedK=strikes)[0],
    x0  = [0.06460112, 2.69262944, 0.08495089],
    bounds = [(-100,100),(0.0001,100),(0.0001,1)],
    method='nelder-mead',
    callback=callback
)

Eval: 0, x=[0.06460112 2.69262944 0.08919843], func: 30383440.484500837
Eval: 1, x=[0.06460112 2.69262944 0.08919843], func: 30383440.484500837
Eval: 2, x=[0.06460112 2.69262944 0.08919843], func: 30383440.484500837
Eval: 3, x=[0.06460112 2.69262944 0.08919843], func: 30383440.484500837
Eval: 4, x=[0.06467755 2.66478898 0.08746358], func: 30371012.18220706
Eval: 5, x=[0.06508615 2.69375483 0.086738  ], func: 30364226.669634614
Eval: 6, x=[0.06497543 2.6748194  0.09064912], func: 30355938.371152185
Eval: 7, x=[0.06475708 2.68520859 0.088741  ], func: 30347622.154908605
Eval: 8, x=[0.06475708 2.68520859 0.088741  ], func: 30347622.154908605
Eval: 9, x=[0.06476176 2.67691788 0.09586929], func: 30317138.1700279
Eval: 10, x=[0.06476176 2.67691788 0.09586929], func: 30317138.1700279
Eval: 11, x=[0.06405684 2.63515805 0.09578543], func: 30299616.083622813
Eval: 12, x=[0.06424556 2.62348946 0.10525819], func: 30284955.63894859
Eval: 13, x=[0.06376833 2.60854622 0.10685647], func: 30266461.2241

KeyboardInterrupt: 

In [122]:
straddlesobs.loc[[12],[120]]

,120
Expiry,
12,91.916301


In [127]:
import warnings
warnings.filterwarnings('ignore')
#Trying to fit just the first price 1M1Y
from scipy.optimize import minimize, basinhopping
EURIBORFIXMARCH22 = 0.03071
Nfeval = 0
def callback(xk):
    global Nfeval
    print(f'Eval: {Nfeval}, x={xk}, func: {VasicekOptimizer(initial=0.03071, mean=xk[0], reversion=xk[1], volatility=xk[2],observedP=straddlesobs, observedK=strikes)[0]}')
    Nfeval += 1
    return True
# callback([0.027832947, 0.11874695, 0.019677])

result = basinhopping(
    # fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observed=straddlesobs)[0],
    func = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observedP=straddlesobs.loc[[12],[120]], observedK=strikes.loc[[12],[120]])[0],
    x0  = [0.06460112, 0.69262944, 0.08495089]#,
    # bounds = [(-100,100),(0.0001,100),(0.0001,1)],
    # method='nelder-mead',
    # callback=callback
)
warnings.filterwarnings('default')

In [128]:
result

                        fun: 4.074611309051606e-16
 lowest_optimization_result:       fun: 4.074611309051606e-16
 hess_inv: array([[0.00589122, 0.06267183, 0.02588957],
       [0.06267183, 0.91967665, 0.25824943],
       [0.02588957, 0.25824943, 0.11499433]])
      jac: array([-1.17344952e-08,  5.60584893e-08,  3.25765746e-08])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 3
     njev: 6
   status: 0
  success: True
        x: array([  6.62287488, 136.76347062,  26.83122804])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 99
                       nfev: 18887
                        nit: 100
                       njev: 4424
                    success: True
                          x: array([  6.62287488, 136.76347062,  26.83122804])

In [129]:
VasicekOptimizer(0.03071, 6.62287488, 136.76347062, 26.83122804,straddlesobs,strikes)[1]

,12,24,36,48,60,84,120,144,180,240,300,360
Expiry,,,,,,,,,,,,
1,107.687756,124.906859,146.243688,120.055751,114.136014,104.781472,103.003132,106.008200,102.566150,213.535523,595.049511,1004.309241
3,108.780807,122.094414,121.811641,115.566549,111.416532,104.273110,102.503396,105.493886,98.075027,226.372199,626.883138,1038.545774
6,104.502919,112.246913,110.805933,106.312567,101.623315,96.474418,97.057354,98.387938,93.268306,267.283508,656.794775,1069.821761
9,100.723581,104.106991,102.065271,98.116593,95.995545,93.102161,93.521964,93.844213,96.950446,308.591151,703.347321,1100.588205
12,96.945783,97.776247,95.140977,93.580695,92.202553,92.042600,91.916301,91.916627,103.631082,335.208194,732.203903,1130.850860
24,89.311748,89.435535,90.145119,92.016515,94.576530,97.069645,93.010513,94.383538,157.446354,452.223515,859.610544,1246.976717
36,86.961223,88.006239,89.604667,92.705382,94.378640,94.277979,92.341668,94.392720,201.336191,534.966320,931.018414,1319.394464
48,84.516048,85.720788,87.566807,89.111008,89.214823,87.107413,89.685976,112.653832,246.793249,599.291498,997.607999,1386.615447
60,82.210168,83.514978,84.524824,84.974437,83.949955,82.804849,85.214305,146.933876,280.630384,659.474303,1074.710714,1431.855101


In [89]:
expiry = 1
tenor  = 12
obsP = straddlesobs[tenor].loc[expiry]
obsK = strikes[tenor].loc[expiry]
def vasiOptVol(initial, mean, rev, vol, expiry, tenor, obsP,obsK):
    print(vol, end='             \r')
    T = np.arange(expiry/12, expiry/12+tenor/12+0.5, 0.5)
    S = np.arange(expiry/12, expiry/12+tenor/12+1)
    Vasi = Vasicek(initial, mean, rev, vol)
    p = Vasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=obsK, payer=True)
    r = Vasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=obsK, payer=False)
    return abs(obsP-((p+r)*10000))

basinhopping(
    func = lambda v: vasiOptVol(0.03071,0.148,10,v,expiry,tenor,obsP,obsK),
    x0 = 0.5
)

                        fun: 35279.20815855801
 lowest_optimization_result:       fun: 35279.20815855801
 hess_inv: array([[0.0023714]])
      jac: array([31.6875])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 161
      nit: 4
     njev: 75
   status: 2
  success: False
        x: array([0.07514477])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 92
                       nfev: 11133
                        nit: 100
                       njev: 5193
                    success: False
                          x: array([0.07514477])